In [ ]:
!python3 -m pip install spacy==2.3.0


# instalacja oficjalnego modelu spaCy
!python3 -m spacy download pl_core_news_lg

# dodatkowe zależności:
!python3 -m pip install tqdm


     |████████████████████████████████| 10.0MB 2.7MB/s 
     |████████████████████████████████| 2.1MB 38.4MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 604.2MB 1.2MB/s 
  Created wheel for pl-core-news-lg: filename=pl_core_news_lg-2.3.0-cp36-none-any.whl size=604232614 sha256=49036871256ac61bb7492efa425da3705b3cf48a0ba1d239bf2803a962fa60b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-b6738qmn/wheels/fb/60/e5/4e03ebc777d2bd26ac5394f00db1695bede7a92bc9e7ce1789
Successfully built pl-core-news-lg
✔ Download and installation successful
You can now load the model via spacy.load('pl_core_news_lg')


# Część szósta - Zastosowanie w filtrze cyberbullyingu

Jednym z głównym zastosowań NLP, jest analiza wydźwięku (sentiment analysis) - automatyczna klasyfikacja tekstów (np. recenzji) ze względu na ich ładunek emocjonalny, wyrażoną ocenę, lub zawartość pewnego "zabarwienia semantycznego". W 2019 roku w PolEvalu, największym konkursie dotyczącym NLP dla języka polskiego, pojawiło się między innymi zadanie dotyczące automatycznej detekcji cyberbullyingu (http://2019.poleval.pl/index.php/tasks/task6).

Skorzystamy z wbudowanych w model embeddingów, aby wytrenować sieć neuronową, służącą do klasyfikacji tweetów. Model będzie zrealizowany w ramach Kerasa, ale spaCy posiada także wbudowane API do tworzenia klasyfikatorów (TextClassifier).

In [ ]:
# POBRANIE DANYCH
!wget http://2019.poleval.pl/task6/task_6-1.zip
!wget http://2019.poleval.pl/task6/task6_test.zip
!unzip task_6-1.zip
!unzip task6_test.zip

--2020-07-14 12:00:54--  http://2019.poleval.pl/task6/task_6-1.zip
Resolving 2019.poleval.pl (2019.poleval.pl)... 213.135.36.94
Connecting to 2019.poleval.pl (2019.poleval.pl)|213.135.36.94|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 339950 (332K) [application/zip]
Saving to: ‘task_6-1.zip’

task_6-1.zip        100%[===================>] 331.98K   687KB/s    in 0.5s    

2020-07-14 12:00:55 (687 KB/s) - ‘task_6-1.zip’ saved [339950/339950]

--2020-07-14 12:00:56--  http://2019.poleval.pl/task6/task6_test.zip
Resolving 2019.poleval.pl (2019.poleval.pl)... 213.135.36.94
Connecting to 2019.poleval.pl (2019.poleval.pl)|213.135.36.94|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70051 (68K) [application/zip]
Saving to: ‘task6_test.zip’

task6_test.zip      100%[===================>]  68.41K   284KB/s    in 0.2s    

2020-07-14 12:00:56 (284 KB/s) - ‘task6_test.zip’ saved [70051/70051]

Archive:  task_6-1.zip
  inflating: training_set_c

In [ ]:
import numpy
import random
import spacy
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Masking, Dropout
from tqdm import tqdm

# funkcje służące przygotowaniu reprezentacji danych
def get_embeddings(vocab):
    max_rank = max(lex.rank for lex in vocab if lex.has_vector)
    vectors = numpy.ndarray((max_rank+1, vocab.vectors_length), dtype='float32')
    for lex in vocab:
        if lex.has_vector:
            vectors[lex.rank] = lex.vector
    return vectors

def get_features(docs, max_length):
    Xs = numpy.zeros((len(docs), max_length), dtype='int32')
    for i, doc in enumerate(docs):
        for j, token in enumerate(doc[:max_length]):
            Xs[i, j] = token.rank if token.has_vector else 0
    return Xs


# wczytywanie modelu
print("Ładowanie modelu.\n")
nlp = spacy.load("pl_core_news_lg")
print("Załadowano model.\n")
print("\n")

# wczytywanie danych
with open("training_set_clean_only_text.txt") as f:
  txt = f.read()
  training_sents = txt.split("\n")[:-1]

with open("training_set_clean_only_tags.txt") as f:
  txt = f.read()
  training_labels = [int(x) for x in txt.split("\n")[:-1]]

with open("Task6/task 01/test_set_clean_only_text.txt") as f:
  txt = f.read()
  test_sents = txt.split("\n")[:-1]

with open("Task6/task 01/test_set_clean_only_tags.txt") as f:
  txt = f.read()
  test_labels = [int(x) for x in txt.split("\n")[:-1]]

train_docs =[]
for x in tqdm(training_sents):
  train_docs.append(nlp(x))
dev_docs = []
for x in tqdm(test_sents):
  dev_docs.append(nlp(x))
max_len = max([len(d) for d in train_docs])

print("\nProporcja wyników dodatnich: {}\n".format(sum(training_labels)/len(training_labels)))

# oversampling
pos_docs = []
for x, y in zip(train_docs, training_labels):
  if y == 1:
    pos_docs.append(x)
oversampling_x = 4*pos_docs
oversampling_y = [1 for x in oversampling_x]
train_docs.extend(oversampling_x)
training_labels.extend(oversampling_y)
indices = list(range(len(train_docs)))
random.shuffle(indices)
train_docs = [train_docs[i] for i in indices]
training_labels = [training_labels[i] for i in indices]

# dane treningowe
train_X = get_features(train_docs, max_len)
train_Y = numpy.zeros((len(training_labels), 2))
for i, l in enumerate(training_labels):
  train_Y[i][l]=1

# dane walidacyjne
dev_X = get_features(dev_docs, max_len)
dev_Y = numpy.zeros((len(test_labels), 2))
for i, l in enumerate(test_labels):
  dev_Y[i][l]=1

for x, y in zip(train_docs[:10], training_labels[:10]):
  print(x, y)
print("\nProporcja wyników dodatnich po oversamplingu: {}\n".format(sum(training_labels)/len(training_labels)))

Using TensorFlow backend.


Ładowanie modelu.



  0%|          | 9/10041 [00:00<02:03, 80.97it/s]

Załadowano model.





100%|██████████| 1000/1000 [00:10<00:00, 92.54it/s]



Proporcja wyników dodatnich: 0.08475251468977194

@anonymized_account @anonymized_account Leczyć można z chorób - nie ze stanu umysłowego IQ :) 1
Cyyyyyk, mamy już zapis dzisiejszej Strefy Kibica z trenerem Ireneuszem Mamrotem. Zapraszamy do słuchania. 😊\nhttps://t.co/RME1zo0xjz 0
Mojej mamie się śniło, że rozjechała mnie ciężarówka rip 0
@anonymized_account Przestań ćpać haszysz, jest życie poza marszem 1
@anonymized_account o widzisz, to nawet nie wiedziałam bo to było jakieś dwa lata temu 0
@anonymized_account Ciekawe w jaki sposób Nurowska okazuje panu profesorowi swoje poddaństwo. Pachnie mi tu sexem oralnym.😁😁😁😁 1
THIS IS WHY I LOVE THE INTERNET 😂😂😂\n\nhttps://t.co/obxKCqhOwH 0
@anonymized_account @anonymized_account @anonymized_account Pewnie z 12 mld całość, ale grosz do grosza 0
@anonymized_account @anonymized_account @anonymized_account Dekompozycja obozu rządzącego stała się faktem 0
@anonymized_account Jak pisdzielstwo 98% wymieniło to co się dziwisz parchu  , polaku złoty

In [ ]:
# przygotowanie modelu Keras
embeddings = get_embeddings(nlp.vocab)
dim = 100
model = Sequential()

model.add(
    Embedding(
        embeddings.shape[0],
        embeddings.shape[1],
        input_length=max_len,
        trainable=False,
        weights=[embeddings]
    )
)
model.add(Masking())
model.add(LSTM(dim))
model.add(Dense(2, activation="softmax"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics = ["accuracy"])


# trening klasyfikatora
model.fit(train_X, train_Y, validation_data=(dev_X,dev_Y), epochs = 4)

Train on 13445 samples, validate on 1000 samples
Epoch 1/4
13445/13445 [==============================] - 54s 4ms/step - loss: 0.4842 - accuracy: 0.7633 - val_loss: 0.3409 - val_accuracy: 0.8490
Epoch 2/4
13445/13445 [==============================] - 51s 4ms/step - loss: 0.2969 - accuracy: 0.8759 - val_loss: 0.3628 - val_accuracy: 0.8480
Epoch 3/4
13445/13445 [==============================] - 52s 4ms/step - loss: 0.1886 - accuracy: 0.9283 - val_loss: 0.4487 - val_accuracy: 0.8720
Epoch 4/4
13445/13445 [==============================] - 51s 4ms/step - loss: 0.1248 - accuracy: 0.9560 - val_loss: 0.5185 - val_accuracy: 0.8640


## Ewaluacja modelu:
W wypadku zadań klasyfikacyjnych binarnych, łatwo można zdefiniować i obliczyć następujące miary:

**precision** (precyzja) - proporcja pozytywnych wyników z modelu, które zostały przypisane prawidłowo

**recall** (czułość) - proporcja pozytywnych przykładów w danych, które zostały poprawnie wykryte

**f1** - średnia harmoniczna precyzji i czułości

In [ ]:
# ewaluacja modelu
preds = model.predict_classes(dev_X)
TP, FP, TN, FN = 0, 0, 0, 0
for s,g in zip(preds, test_labels):
  if s == 1 and g == 1:
    TP +=1
  elif s ==1:
    FP +=1
  elif g ==1:
    FN +=1
  else:
    TN +=1

precision = TP/(TP+FP)
recall = TP/(TP+FN)
f1 = 2* ((precision * recall)/(precision + recall))
print("precision: {}, recall: {}, f1: {}".format(precision, recall, f1))

test_example = "To jest odrażające, nie wiem po co w ogóle żyjesz."
test_x = get_features([nlp(test_example)], max_len)
print(model.predict_classes(test_x)[0])

precision: 0.4864864864864865, recall: 0.26865671641791045, f1: 0.3461538461538462
0
